### Utility functions

_Copied in to avoid namespace issues. Should just do an import all... Fix it later..._

In [1035]:
def check_file_age(filename, threshold=30, data_path='assets/'):
	threshold *= 86_400 # Convert days to seconds
	file_path = f"{data_path}{filename}"
	if os.path.exists(file_path):
		print(f">> File  [ {filename} ] found. Checking age... ")
		st=os.stat(file_path)
		mtime=time.time() - st.st_mtime
		if mtime > threshold:
			print(f"\t+ File [ {filename} ] exceeds age threshold. Refresh...")
			return False
		else:
			print(f"\t+ File  [ {filename} ] not expired. Load from disk...")
			return True
	else:
		print(f">> File [ {filename} ] not found. Fetching...")
		return False

### Country code stuff

In [1095]:
def fetch_eu_states(add_efta=True):
    cc = coco.CountryConverter()
    efta_states = [{'ISO3':'NOR','name_short':'Norway','ISO2':'NO'},
               {'ISO3':'ISL','name_short':'Iceland','ISO2':'IS'},
               {'ISO3':'CHE','name_short':'Switzerland','ISO2':'CH'},
              ]
    eu_states = cc.data.loc[cc.data.EU <= 2017][['ISO3', 'ISO2', 'name_short']]
    if add_efta:
        eu_states = eu_states.append(efta_states, ignore_index=True)
    eu_states.columns = ['ISO3166_a3','ISO3166_a2','CountryName']
    return eu_states
    

In [1037]:
cc = coco.CountryConverter()

eu_states = set(cc.data.loc[cc.data.EU <= 2017]['ISO3'])
oecd_states = set(cc.data.loc[cc.data.OECD <= 2017]['ISO3'])
eu_states - oecd_states

{'BGR', 'CYP', 'HRV', 'LTU', 'MLT', 'ROU'}

In [1038]:
# Union of OECD / EU states
# cc.data.loc[(cc.data.EU.notnull()) | (cc.data.OECD.notnull()),['EU', 'OECD', 'name_short', 'ISO3']]

### Eurostat stuff

In [271]:
import eurostat
import country_converter as coco
import time


##### Notes:

Metadata from here [ https://ec.europa.eu/eurostat/cache/metadata/en/hlth_sha11_esms.htm  ]
Python API [ https://pypi.org/project/eurostat/ ]
EuroStat Database overview [ https://ec.europa.eu/eurostat/data/database ]
Explanations from the EIONET Data Dictionary [ http://dd.eionet.europa.eu/vocabularyconcept/eurostat/unit/EUR_HAB/view ]
and [ https://ec.europa.eu/eurostat/statistics-explained/index.php/Tutorial:Symbols_and_abbreviations ]


This indicator, "Health care expenditure by provider (hlth_sha11_hp) returns a dataframe with 29 columns. The first three columnns, `unit`, `icha11_hp` and `geo\time` are categorical columns. The next 26 columns are data aggregated by year.

The `unit` column specifies the unit of the column. Map of `unit` codes to categories:
+ `MIO_EUR` expenditure amount in millions of euro
+ `MIO_NAC` expenditure amount in millions of national currency
+ `MIO_PPS` expenditure amount in millions of PPS
+ `PC_GDP` percentage of GDP
+ `EUR_HAB` amount in euro per capita
+ `NAC_HAB` amount in national currency per capita 
+ `PPS_HAB` amount in PPS per capita
+ `PC_CHE` percentage of current health expenditure (CHE)

The `icha11_hp` column indicates the health care provider. There are 35 options for this, but I think the only thing I'm interested in at the moment is `TOTAL`

The `geo\time` column indicates the geographic entity and includes 33 entities, the EU28 and the following five additional countries. 
+ BA (Bosnia and Herzegovina)
+ CH (Switzerland)
+ IS (Iceland)
+ LI (Lichtenstein)
+ NO (Norway)

Four of these for the EFTA and overlaps heavily with the EU in various treaties. Bosnia, I assume is included because of expected future membership.

In addition, the European Commission uses the following non-standard abbreivations, which needs to be included as an additional map.
+ EL instead of GR for Greece
+ UK instead of GB for the United Kingdom

I want the 2017 and ten year median number in expenditure per capita and expenditure as a percentage of GDP.

I decided to just use 2016 for this, more complicated features would require more work...

The eurostat data that I'm using is:
+ Health statistics:
    + `sdg_08_10` Real GDP per capita
+ Economic statistics:
    + `tec00127` General government budget deficit in %GDP
    + `sdg_17_40` Total government debt in %GDP
    + `hlth_sha11_hp` In per capita spending and %GDP (I think these will be correlated so will use only one)
+ Demographic statistics:
    + `tps00003` Population density
    + `tps00028` Proportion of population aged 65 and over
    + `tesov190` Household distribution
    + `tps00001` Population on January 1st of year


In [1285]:
def fetch_economic_indicator(code, name, year, unit=None, unit_name='unit'):
    full_df = eurostat.get_data_df(code, flags=False)
    geo_entities = full_df['geo\\time'].unique()[:-7]
    if unit is not None:
        full_df = full_df.loc[(full_df[unit_name] == unit) &
                (full_df['geo\\time'].isin(geo_entities))]
    else:
        full_df = full_df.loc[(full_df['geo\\time'].isin(geo_entities))]
    indicator_name = f"EUS_{name}_{year}"

    full_df = full_df[['geo\\time', year]]
    full_df.columns = ['ISO3166_a2', indicator_name]
    full_df.loc[full_df['ISO3166_a2'] == 'EL', 'ISO3166_a2'] = 'GR'
    full_df.loc[full_df['ISO3166_a2'] == 'UK', 'ISO3166_a2'] = 'GB'
    return full_df

In [1236]:
def fetch_health_exp(indicators):
    #Working rewrite of health care stats

    for code in indicators.keys():
        full_df = eurostat.get_data_df(code, flags=False)
        geo_entities = full_df['geo\\time'].unique()[:-7]
        units = indicators[code]['units']
        providers = indicators[code]['providers']
        indicator = indicators[code]['indicator']

        indicator_df = full_df.loc[(full_df['unit'].isin(units)) &
                                  (full_df[indicator].isin(providers)) &
                                  (full_df['geo\\time'].isin(geo_entities))].copy()

        year = indicators[code]['year']

        final_df = fetch_eu_states()['ISO3166_a2']
        for unit in units:
            indicator_name = f"EUS_{indicators[code]['name']}_{str.lower(unit)}_{year}"
            subset_df = indicator_df.loc[indicator_df['unit']==unit, ['geo\\time', year]]
            subset_df.columns = ['ISO3166_a2', indicator_name]
            # Fix Greece and the UK
            subset_df.loc[subset_df['ISO3166_a2'] == 'EL', 'ISO3166_a2'] = 'GR'
            subset_df.loc[subset_df['ISO3166_a2'] == 'UK', 'ISO3166_a2'] = 'GB'
            final_df = pd.merge(final_df, subset_df, how='left', on='ISO3166_a2')

    #final_df.drop('ISO3166_a2', axis=1, inplace=True)
    return final_df

In [1263]:
def fetch_eurostat_indicators(save_file=True,
                                threshold=30,
                                data_path='assets/',
                                filename='Eurostat_indicators_EU.csv'):
    if check_file_age(filename, threshold=threshold):
        return pd.read_csv(f'{data_path}{filename}')
    
    indicators = {'hlth_sha11_hp':{'name':'EUS_healthExp',
                               'year':2016,
                               'units':['PC_GDP','EUR_HAB'],
                               'providers':['TOTAL'],
                               'indicator':'icha11_hp'}}

    health_exp = fetch_health_exp(indicators) # This one has is a little more complicated...

    gdp_per_capita = fetch_economic_indicator('sdg_08_10', 'EUS_population', 2019, 'CLV10_EUR_HAB')
    govt_budget_surplus = fetch_economic_indicator('tec00127', 'EUS_govt_budget_surplus_pct_gdp', 2019, 'PC_GDP')
    govt_debt = fetch_economic_indicator('sdg_17_40', 'EUS_govt_debt_pct_gdp', 2019, 'PC_GDP')
    population_density = fetch_economic_indicator('tps00003', 'EUS_population_density', 2018, 'PER_KM2')
    propOver65 = fetch_economic_indicator('tps00028', 'EUS_popOver65_pct', 2019)
    singleHouseholds_pct = fetch_economic_indicator('tesov190', 'EUS_singleHouseholds_pct', 2018, 'A1', 'hhtyp')
    population = fetch_economic_indicator('tps00001', 'EUS_pop_mil', 2019, unit=None)

    final_df = fetch_eu_states()

    final_df = pd.merge(final_df, health_exp, how='left', on='ISO3166_a2')
    final_df = pd.merge(final_df, gdp_per_capita, how='left', on='ISO3166_a2')
    final_df = pd.merge(final_df, govt_budget_surplus, how='left', on='ISO3166_a2')
    final_df = pd.merge(final_df, govt_debt, how='left', on='ISO3166_a2')
    final_df = pd.merge(final_df, population_density, how='left', on='ISO3166_a2')
    final_df = pd.merge(final_df, propOver65, how='left', on='ISO3166_a2')
    final_df = pd.merge(final_df, singleHouseholds_pct, how='left', on='ISO3166_a2')
    final_df = pd.merge(final_df, population, how='left', on='ISO3166_a2')
    
    final_df.drop('ISO3166_a2', axis=1, inplace=True)
    
    if save_file:
        final_df.to_csv(f"{data_path}{filename}", index=False)
    
    return final_df

In [1264]:
final_df = fetch_eurostat_indicators()

>> File [ Eurostat_indicators_EU.csv ] not found. Fetching...


In [1265]:
final_df

,ISO3166_a3,CountryName,EUS_EUS_healthExp_pc_gdp_2016,EUS_EUS_healthExp_eur_hab_2016,EUS_EUS_population_2019,EUS_EUS_govt_budget_surplus_pct_gdp_2019,EUS_EUS_govt_debt_pct_gdp_2019,EUS_EUS_population_density_2018,EUS_EUS_popOver65_pct_2019,EUS_EUS_singleHouseholds_pct_2018,EUS_EUS_pop_mil_2019
0,AUT,Austria,10.42,4248.37,38250.0,0.7,70.4,107.1,18.8,16.8,8858775.0
1,BEL,Belgium,10.30,3861.23,35900.0,-1.9,98.6,375.3,18.9,15.3,11455519.0
2,BGR,Bulgaria,8.23,555.64,6800.0,2.1,20.4,63.9,21.3,13.8,7000039.0
3,HRV,Croatia,6.83,763.03,12480.0,0.4,73.2,73.2,20.6,8.9,4076246.0
4,CYP,Cyprus,6.79,1474.00,24250.0,1.7,95.5,94.4,16.1,7.8,875899.0
5,CZE,Czech Republic,7.15,1193.39,18000.0,0.3,30.8,137.7,19.6,12.2,10649800.0
6,DNK,Denmark,10.18,5014.00,49190.0,3.7,33.2,138.0,19.6,22.2,5806081.0
7,EST,Estonia,6.50,1071.71,15670.0,-0.3,8.4,30.4,19.8,18.6,1324820.0
8,FIN,Finland,9.43,3707.20,37170.0,-1.1,59.4,18.1,21.8,21.5,5517919.0
9,FRA,France,11.48,3843.67,33270.0,-3.0,98.1,105.6,20.1,16.5,67012883.0


In [1237]:
# def fetch_health_indicators(year_range = list(range(2007,2018)),
#                             save_file=True,
#                               threshold=30,
#                               data_path='assets/',
#                               filename='eurostat_health.csv'):
#     if check_file_age(filename, threshold=threshold):
#         return pd.read_csv(f'{data_path}{filename}')

#     code = 'hlth_sha11_hp'
#     full_health_df = eurostat.get_data_df(code, flags=False)
#     geo_entities = list(full_df['geo\\time'].unique()[:-7][:-7]) + ['EU_28']
#     units = ['PC_GDP','EUR_HAB']
#     providers = ['TOTAL']

#     subset_df = full_health_df.loc[(full_health_df['unit'].isin(units)) &
#                               (full_health_df['icha11_hp'].isin(providers)) &
#                               (full_health_df['geo\\time'].isin(geo_entities))].copy()

#     subset_df['mean10y'] = subset_df[year_range].mean(axis=1)
#     subset_df['med10y'] = subset_df[year_range].median(axis=1)

#     subset_df = subset_df[['unit', 'geo\\time', 2017, 'mean10y', 'med10y']]
    
#     pivot_df = subset_df.pivot(index='geo\\time', columns='unit', values=[2017, 'mean10y', 'med10y'])
#     pivot_df = pivot_df.reset_index()
    
#     pivot_df.columns = ['IS03166_a2', 'EUR_HAB_2017', 'PC_GDP_2017',
#                         'EUR_HAB_mean10y', 'PC_GDP_mean10y',
#                         'EUR_HAB_med10y', 'PC_GDP_med10y']

#     # Correct Greece and the UK to ISO3166:2
#     pivot_df.loc[pivot_df['IS03166_a2'] == 'EL', 'IS03166_a2'] = 'GR'
#     pivot_df.loc[pivot_df['IS03166_a2'] == 'UK', 'IS03166_a2'] = 'GB'

#     cc = coco.CountryConverter()
#     eu_states = fetch_eu_states()
#     eu_states.columns = ['IS03166_a3', 'IS03166_a2', 'CountryName']
# #     iso2_map = cc.data.loc[cc.data.EU <= 2017][['ISO3','ISO2']]
# #     iso2_map.columns = ['IS03166_a3', 'IS03166_a2']
    
#     pivot_df = pd.merge(eu_states, pivot_df, how="inner",
#                         on='IS03166_a2')
#     pivot_df.drop('IS03166_a2', inplace=True, axis=1)
    
#     if save_file:
#         pivot_df.to_csv(f"{data_path}{filename}", index=False)
        
#     return pivot_df, subset_df

In [1281]:
full_df = eurostat.get_data_df('hlth_sha11_hp', flags=False)

array(['AT', 'BA', 'BE', 'BG', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL',
       'ES', 'FI', 'FR', 'HR', 'HU', 'IE', 'IS', 'IT', 'LI', 'LT', 'LU',
       'LV', 'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'UK'],
      dtype=object)

In [1107]:
# # Working rewrite of health care stats
# final_df = fetch_eu_states()
# for code in indicators.keys():
#     full_df = eurostat.get_data_df(code, flags=False)
#     geo_entities = full_df['geo\\time'].unique()[:-7]
#     units = indicators[code]['units']
#     providers = indicators[code]['providers']
#     indicator = indicators[code]['indicator']

#     indicator_df = full_df.loc[(full_df['unit'].isin(units)) &
#                               (full_df[indicator].isin(providers)) &
#                               (full_df['geo\\time'].isin(geo_entities))].copy()
    
#     year = indicators[code]['year']

#     for unit in units:
#         indicator_name = f"EUS_{indicators[code]['name']}_{str.lower(unit)}_{year}"
#         subset_df = indicator_df.loc[indicator_df['unit']==unit, ['geo\\time', year]]
#         subset_df.columns = ['ISO3166_a2', indicator_name]
#         # Fix Greece and the UK
#         subset_df.loc[subset_df['ISO3166_a2'] == 'EL', 'ISO3166_a2'] = 'GR'
#         subset_df.loc[subset_df['ISO3166_a2'] == 'UK', 'ISO3166_a2'] = 'GB'
#         final_df = pd.merge(final_df, subset_df, how='left', on='ISO3166_a2')

#     final_df.drop('ISO3166_a2', axis=1, inplace=True)

In [994]:
# Old version, took averages
# def fetch_health_indicators(year_range = list(range(2007,2018)),
#                             save_file=True,
#                               threshold=30,
#                               data_path='assets/',
#                               filename='eurostat_health.csv'):
#     if check_file_age(filename, threshold=threshold):
#         return pd.read_csv(f'{data_path}{filename}')

#     code = 'hlth_sha11_hp'
#     full_health_df = eurostat.get_data_df(code, flags=False)
#     geo_entities = list(full_df['geo\\time'].unique()[:-7]) + ['EU_28']
#     units = ['PC_GDP','EUR_HAB']
#     providers = ['TOTAL']

#     subset_df = full_health_df.loc[(full_health_df['unit'].isin(units)) &
#                               (full_health_df['icha11_hp'].isin(providers)) &
#                               (full_health_df['geo\\time'].isin(geo_entities))].copy()

#     subset_df['mean10y'] = subset_df[year_range].mean(axis=1)
#     subset_df['med10y'] = subset_df[year_range].median(axis=1)

#     subset_df = subset_df[['unit', 'geo\\time', 2017, 'mean10y', 'med10y']]
    
#     pivot_df = subset_df.pivot(index='geo\\time', columns='unit', values=[2017, 'mean10y', 'med10y'])
#     pivot_df = pivot_df.reset_index()
    
#     pivot_df.columns = ['IS03166_a2', 'EUR_HAB_2017', 'PC_GDP_2017',
#                         'EUR_HAB_mean10y', 'PC_GDP_mean10y',
#                         'EUR_HAB_med10y', 'PC_GDP_med10y']

#     # Correct Greece and the UK to ISO3166:2
#     pivot_df.loc[pivot_df['IS03166_a2'] == 'EL', 'IS03166_a2'] = 'GR'
#     pivot_df.loc[pivot_df['IS03166_a2'] == 'UK', 'IS03166_a2'] = 'GB'

#     cc = coco.CountryConverter()
#     eu_states = fetch_eu_states()
#     eu_states.columns = ['IS03166_a3', 'IS03166_a2', 'CountryName']
# #     iso2_map = cc.data.loc[cc.data.EU <= 2017][['ISO3','ISO2']]
# #     iso2_map.columns = ['IS03166_a3', 'IS03166_a2']
    
#     pivot_df = pd.merge(eu_states, pivot_df, how="inner",
#                         on='IS03166_a2')
#     pivot_df.drop('IS03166_a2', inplace=True, axis=1)
    
#     if save_file:
#         pivot_df.to_csv(f"{data_path}{filename}", index=False)
        
#     return pivot_df, subset_df

### OECD Health care statistics

In [946]:
import requests
import pandas as pd
import bs4

#import json
#import requests as rq
#import re

#### OECD Notes

The overivew of the OECD API can be found here [ https://data.oecd.org/api/sdmx-json-documentation/ ]. I found it a little confusing, but the initial question and the first two answers from this Stack Exchange answer proved really helpful, [ https://stackoverflow.com/questions/40565871/read-data-from-oecd-api-into-python-and-pandas ]. The below section on Using the OECD API explains what I understand of how it works.


#### OECD exploration functions

In [447]:
def get_keyFamilies(save=True,
                   data_path="assets/",
                   filename="OECD_keyFamilies.csv"):    
    dataStructure_url = "https://stats.oecd.org/restsdmx/sdmx.ashx/GetDataStructure/ALL/all?format=SDMX-ML"
    r = requests.get(dataStructure_url)
    soup = bs4.BeautifulSoup(r.content, 'xml')
    keyFamilies = soup.find_all('KeyFamily')
    
    dict_list = []
    for kf in keyFamilies:
        kf_dict = {}
        kf_dict['KeyFamily'] = kf.get('id')
        kf_dict['Agency'] = kf.get('agencyID')
        for n in kf.find_all('Name'):
            if n.get('xml:lang') == "en":
                kf_dict['Name'] = n.get_text()
        dict_list.append(kf_dict)
    
    kf_df = pd.DataFrame(dict_list)
    if save:
        kf_df.to_csv(f"{data_path}{filename}", index=False)
    return kf_df

In [837]:
def get_kf_details(keyFamily,
                   save=True,
                  data_path="assets/",
                   filename=None):
    if filename == None:
        filename = f"OECD_{keyFamily}_details.csv"
    structure_req_url = "https://stats.oecd.org/restsdmx/sdmx.ashx/GetDataStructure/"
    req_url = structure_req_url + keyFamily
    r = requests.get(req_url)

    soup = bs4.BeautifulSoup(r.content, 'xml')

    codeLists = soup.find_all('CodeList')

    dict_list = []

    for cl in codeLists:
        codes = cl.find_all('Code')
        for c in codes:
            code_dict = {} 
            code_dict['code_list']  = cl.get('id')
            code_dict['value']  = c.get('value')
            for d in c.find_all('Description'):
                if d.get('xml:lang') == "en":
                    code_dict['description']  = d.get_text()
            dict_list.append(code_dict)
    kf_details_df = pd.DataFrame(dict_list)
    if save:
        kf_details_df.to_csv(f"{data_path}{filename}", index=False)
    return kf_details_df



In [868]:
OECD_ROOT_URL = "http://stats.oecd.org/SDMX-JSON/data"

def make_OECD_request(keyFamily, dimensions, params = None, root_dir = OECD_ROOT_URL):
    """ Make URL for the OECD API and return a response
        4 dimensions: location, subject, measure, frequency
        
        Adapted from [ https://stackoverflow.com/questions/40565871/read-data-from-oecd-api-into-python-and-pandas ]
    """

    param_str = "?contentType=csv"
    if params is not None:
        for key in params.keys():
            param_str += f"&{key}={params[key]}"

    dim_args = ['+'.join(d) for d in dimensions]
    dim_str = '.'.join(dim_args)

    url = f"{root_dir}/{keyFamily}/{dim_str}/all{param_str}"
    #url = root_dir + '/' + dsname + '/' + dim_str + '/all?contentType=csv&startTime=2010-Q2&endTime=2011-Q4'

    print(f'Requesting URL [{url}]')
    return pd.read_csv(url)
    #return rq.get(url = url, params = params)


In [617]:
def fetch_single_indicator_oecd(indicator,
                                unit='',
                                countries='',
                                start_year=2010,
                                end_year=2020):
    
    json_req = f"HEALTH_REAC/{indicator}..{countries}/all"
    additional_parameters = f"&startTime={start_year}-Q1&endTime={end_year}-Q4&dimensionAtObservation=allDimensions"
    request_df = get_from_oecd(json_req,additional_parameters)
    return request_df

In [827]:
def check_indicator_units(indicators):
    request_df = get_oecd_indicators(indicators)
    for key in indicators.keys():
        subframe = request_df.loc[request_df['VAR']==key]
        print(f"{key} : {subframe.shape[0]}\t{subframe.UNIT.unique()}")

#### Using the OECD API

In [842]:
# Working example of direct API call
df = pd.read_csv("http://stats.oecd.org/sdmx-json/data/QNA/.GDP+B1_GE.CUR+VOBARSA.Q/all?startTime=2009-Q1&endTime=2011-Q4&contentType=csv")
# df

In [ ]:
# This uses the SDMX database to get all of the available datasets and 
# save them to a CSV file.
keyFamilies_df = get_keyFamilies()

In [838]:
# For a given key family, this gets the list of all associated indicators
indicator_df = get_kf_details("HEALTH_REAC")
indicator_df.code_list.unique()

In [861]:
# This gets all of a dataset, it is really slow and may hit the row limit
# oecd_url = f"http://stats.oecd.org/sdmx-json/data/{keyFamily}/all/all?contentType=csv"
# df = pd.read_csv(oecd_url)

In [862]:
# Instead, by limiting the years, you can still get the columns
keyFamily = 'HEALTH_STAT'
year = '2011'
oecd_url = f"http://stats.oecd.org/sdmx-json/data/{keyFamily}/all/all?contentType=csv&startTime={year}-Q1&endTime={year}-Q4"
df = pd.read_csv(oecd_url)
print(df.columns)

Index(['VAR', 'Variable', 'UNIT', 'Measure', 'COU', 'Country', 'YEA', 'Year',
       'Value', 'Flag Codes', 'Flags'],
      dtype='object')


In [945]:
# Then, you use all of the columns with capital letters as variables, which 
# are separated with '.' symbols except for 'YEA' or something similar which
# indicates time

# e.g. for HEALTH_STAT, there are three columns
country = 'USA'
var = ''
unit = ''
oecd_url = f"http://stats.oecd.org/sdmx-json/data/{keyFamily}/{var}.{unit}.{country}/all?contentType=csv&startTime={year}-Q1&endTime={year}-Q4"
df = pd.read_csv(oecd_url)
# This can then be used to give you the variables
print(df['VAR'].unique()[0:3])

['DISASRAB' 'EVIEFE00' 'EVIEFE65']


In [864]:
# Then, for a given variable, you can get the units:
country = 'USA'
var = 'EVIEHO80'
unit = ''
oecd_url = f"http://stats.oecd.org/sdmx-json/data/{keyFamily}/{var}.{unit}.{country}/all?contentType=csv&startTime={year}-Q1&endTime={year}-Q4"
df = pd.read_csv(oecd_url)
df['UNIT'].unique()

array(['EVIDUREV', 'EVIHFEEV'], dtype=object)

In [866]:
# To request multiple values of a variable, join them with '+' symbols
country_list = ['USA', 'ESP', 'NOR']
country_str = '+'.join(country_list)
var_list = ['EVIEHO80', 'COMDIMEA']
var_str = '+'.join(var_list)
# I think a unit has to exist, so either don't specify the unit or 
# select a single variable at a time
unit_str = ''
oecd_url = f"http://stats.oecd.org/sdmx-json/data/{keyFamily}/{var_str}.{unit_str}.{country_str}/all?contentType=csv&startTime={year}-Q1&endTime={year}-Q4"
df = pd.read_csv(oecd_url)
df

,VAR,Variable,UNIT,Measure,COU,Country,YEA,Year,Value,Flag Codes,Flags
0,EVIEHO80,Males at age 80,EVIDUREV,Years,NOR,Norway,2011,2011,5.6,NaN,NaN
1,EVIEHO80,Males at age 80,EVIDUREV,Years,ESP,Spain,2011,2011,7.8,NaN,NaN
2,EVIEHO80,Males at age 80,EVIDUREV,Years,USA,United States,2011,2011,8.2,NaN,NaN
3,COMDIMEA,Incidence of measles,PERCMTTX,Incidence per 100 000 population,NOR,Norway,2011,2011,0.8,NaN,NaN
4,COMDIMEA,Incidence of measles,PERCMTTX,Incidence per 100 000 population,ESP,Spain,2011,2011,7.5,NaN,NaN
5,COMDIMEA,Incidence of measles,PERCMTTX,Incidence per 100 000 population,USA,United States,2011,2011,0.1,NaN,NaN
6,EVIEHO80,Males at age 80,EVIHFEEV,Difference male-female (years),NOR,Norway,2011,2011,-4.2,NaN,NaN
7,EVIEHO80,Males at age 80,EVIHFEEV,Difference male-female (years),ESP,Spain,2011,2011,-3.1,NaN,NaN
8,EVIEHO80,Males at age 80,EVIHFEEV,Difference male-female (years),USA,United States,2011,2011,-1.5,NaN,NaN


In [943]:
# This was an earlier version that uses a simple request thing.
# Next refactor, merge this through.
#json_req = "QNA/AUT.PHYS+B1_GE.CUR+VOBARSA.Q/all"
json_req = "HEALTH_REAC/all/all"
additional_parameters = "" #"&startTime=2010-Q2&endTime=2011-Q4&dimensionAtObservation=allDimensions"
request_df = get_from_oecd(json_req,additional_parameters)
# request_df

Having determined which variables you want to use, you can then create the queries you actually care about.

#### OECD databases of interest

In [844]:
# Potentially interesting / useful datasets
datasets = ["PARTNER", # Trade in value by partner countries
            "HEALTH_STAT", # Health Status
            "HEALTH_REAC", # Health Care Resources
                                # PHYS Physicians /  PAGGTOPY Total physicians
                                # MINU Nurses 
                                # Total Hospital beds HOPITBED
            "HEALTH_PROC", # Health Care Utilisation
            "HEALTH_LTCR", # Long-Term Care Resources and Utilisation
            "HEALTH_HCQI", # Health Care Quality Indicators
            "HEALTH_LVNG", # Non-Medical Determinants of Health
            "HEALTH_DEMR", # Demographic References
            "HEALTH_ECOR", # Economic References
            "SHA", # Health expenditure and financing
            "EBDAG", # Expenditure by disease, age and gender under the System of Health Accounts (SHA) Framework
            "HEALTH_WFMI", # Health Workforce Migration
           ]

#### OECD Health Indicators

In [869]:
def get_from_oecd(sdmx_query,
                  additional_parameters=""):
    df = pd.read_csv(f"https://stats.oecd.org/SDMX-JSON/data/{sdmx_query}?contentType=csv{additional_parameters}")
    return df

In [1040]:
def get_oecd_indicators(indicators,
                        save_file=True,
                        threshold=30,
                        data_path='assets/',
                        filename='OECD_healthIndicators_EU.csv'):
    if check_file_age(filename, threshold=threshold):
        return pd.read_csv(f'{data_path}{filename}')

    # Create list of countries that are in the EU and OECD
    cc = coco.CountryConverter()
    eu_states = fetch_eu_states()['ISO3166_a3']
    oecd_states = set(cc.data.loc[cc.data.OECD <= 2017]['ISO3'])
    eu_oecd_overlap = oecd_states.intersection(eu_states)
    country_str = "+".join(eu_oecd_overlap)
    
    # Request the indicators
    request_df = pd.DataFrame()
    for key in indicators.keys():
        json_req = f"HEALTH_REAC/{key}.{indicators[key]['unit']}.{country_str}/all"
        additional_parameters = "&startTime=2017-Q1&endTime=2017-Q4&dimensionAtObservation=allDimensions"
        temp_df = get_from_oecd(json_req,additional_parameters)
        request_df = pd.concat([request_df, temp_df])
    
    # Funge the resulting dataframe into the desired shape
    indicator_df = fetch_eu_states()[['ISO3166_a3', 'CountryName']]
    #indicator_df.columns = ['ISO3166_a3', 'CountryName']
    for idx, indicator in enumerate(request_df.VAR.unique()):
        subset_df = request_df.loc[request_df.VAR == indicator]
        name = indicators[indicator]['name']
        year = subset_df['Year'].unique()[0]
        name = f'OECD_{name}_{year}'
        subset_df = subset_df[['COU', 'Value']]
        subset_df.columns = ['ISO3166_a3', name]
        indicator_df = pd.merge(indicator_df, subset_df, how='left', on='ISO3166_a3')

    if save_file:
        indicator_df.to_csv(f"{data_path}{filename}", index=False)
        
    return indicator_df

In [1041]:
indicators = {'PAGGTOPY':{'unit':'PERSMYNB','name':'Phys_total'},
              'MINUINFI':{'unit':'PERSMYNB','name':'Nurse_total'},
              'HOPITBED':{'unit':'NOMBRENB','name':'HospBed_total'},} 

oecd_indicators = get_oecd_indicators(indicators)
oecd_indicators

>> File [ OECD_healthIndicators_EU.csv ] not found. Fetching...


,ISO3166_a3,CountryName,OECD_Phys_total_2017,OECD_Nurse_total_2017,OECD_HospBed_total_2017
0,AUT,Austria,45596.0,60263.0,64805.00
1,BEL,Belgium,35069.0,NaN,64423.00
2,BGR,Bulgaria,NaN,NaN,NaN
3,HRV,Croatia,NaN,NaN,NaN
4,CYP,Cyprus,NaN,NaN,NaN
5,CZE,Czech Republic,NaN,85372.0,70214.00
6,DNK,Denmark,NaN,NaN,15037.00
7,EST,Estonia,4569.0,8157.0,6185.00
8,FIN,Finland,NaN,NaN,18071.90
9,FRA,France,211162.0,NaN,399865.00


#### Exploring the variables

In [937]:
physician_indicators = {    'PHYSMEDE':'PERSMYNB',
                            'PHYSPAPS':'PERSMYNB',
                            'PHYSREGP':'PERSMYNB',
                            'PAGGTOPY':'PERSMYNB',
#                             'MINUPANU':'',
#                             'MINUPAPN':'',
#                             'MINUPAAP':'',
#                             'MINUNULP':'',
#                             'MINULPPN':'',
#                             'MINULPAP':'',
             }
# check_indicator_units(physician_indicators) # I changed this function, need to update...

In [939]:
nursing_indicators = {  'MINUINFI':'PERSMYNB',
                'MINUQUAL':'PERSMYNB',
                'MINUASSO':'PERSMYNB',
                'MINUPANU':'PERSMYNB',
                'MINUPAPN':'PERSMYNB',
                'MINUPAAP':'PERSMYNB',
                'MINUNULP':'PERSMYNB',
                'MINULPPN':'PERSMYNB',
                'MINULPAP':'PERSMYNB',
             }

# check_indicator_units(nursing_indicators) # I changed this function, need to update...

In [1042]:
cc = coco.CountryConverter()

eu_states = fetch_eu_states()['ISO3166_a3']
oecd_states = set(cc.data.loc[cc.data.OECD <= 2017]['ISO3'])
eu_oecd_overlap = oecd_states.intersection(eu_states)

country_str = "+".join(eu_oecd_overlap)
country_str

_ = fetch_single_indicator_oecd('MINUQUAL',countries=country_str, unit='PERSMYNB', start_year=2017, end_year=2017)

### Hofstede Cultural Dimensions

In [1043]:
import pandas as pd
import os
import requests
from fake_useragent import UserAgent
from io import StringIO
import sys

In [1044]:
# In case fake_useragent isn't found:
# !{sys.executable} -m pip install fake_useragent

#### Notes
Dataset source [https://www.hofstede-insights.com/country-comparison/france,germany,spain,the-uk/]

From [ https://www.researchgate.net/post/Are_there_any_alternatives_for_the_dimensions_of_national_culture ]

_See Myers, M. D. and F. B. Tan (2002). "Beyond Models of National Culture in Information Systems Research." Journal of Global Information Management 10(2).
There is an interesting debate on this: Baskerville, R. F. (2003). "Hofstede never studied culture." Accounting, Organizations and Society 28(1): 1-14. Hofstede replied with Hofstede, G. (2003). "What is culture? A reply to Baskerville." Accounting, Organizations and Society 28: 811-813._

In [1048]:
def load_cultural_dimensions_data(save_file=True,
                                  threshold=30,
                                  data_path='assets/',
                                  filename='HCD_culturalDimensions_EU.csv'):
    if check_file_age(filename, threshold=threshold):
        return pd.read_csv(f'{data_path}{filename}')
    
    hofstede_url = "https://geerthofstede.com/wp-content/uploads/2016/08/6-dimensions-for-website-2015-08-16.csv"
    ua_str = UserAgent().chrome
    r = requests.get(hofstede_url, headers={"User-Agent": ua_str})
    hofstede_df = pd.read_csv(StringIO(r.content.decode('utf-8')),
                                sep=";",na_values="#NULL!")

    #ISO3_df = cc.data.loc[cc.data.EU <= 2017][['ISO3','name_short']]
    eu_states = fetch_eu_states()
    
    # Some countries are wrong, so fix manually...
    # It would be an idea to create a Hofstede to ISO3 code map
    country_pairs = {'Slovak Rep': 'Slovakia', 
                 'Czech Rep' : 'Czech Republic',
                 'Great Britain' : 'United Kingdom'}
    for key in country_pairs.keys():
        hofstede_df.loc[hofstede_df['country'] == key,
                            'country'] = country_pairs[key]

    hofstede_df = pd.merge(eu_states, hofstede_df, how='left',
                           left_on="CountryName", right_on="country")
    hofstede_df = hofstede_df[['ISO3166_a3', 'CountryName', 'pdi',
                               'idv', 'mas', 'uai', 'ltowvs', 'ivr']]

    hofstede_df.columns = ['ISO3166_a3',
                           'CountryName',
                           'HCD_PowerDistance',
                           'HCD_Individualism',
                           'HCD_Masculinity',
                           'HCD_UncertaintyAvoidance',
                           'HCD_LongTermOrientation',
                           'HCD_Indulgence']
    if save_file:
        hofstede_df.to_csv(f"{data_path}{filename}", index=False)
    
    return hofstede_df


In [1049]:
hofstede_df = load_cultural_dimensions_data()
hofstede_df

>> File [ HCD_culturalDimensions_EU.csv ] not found. Fetching...


,ISO3166_a3,CountryName,HCD_PowerDistance,HCD_Individualism,HCD_Masculinity,HCD_UncertaintyAvoidance,HCD_LongTermOrientation,HCD_Indulgence
0,AUT,Austria,11.0,55.0,79.0,70.0,60.0,63.0
1,BEL,Belgium,65.0,75.0,54.0,94.0,82.0,57.0
2,BGR,Bulgaria,70.0,30.0,40.0,85.0,69.0,16.0
3,HRV,Croatia,73.0,33.0,40.0,80.0,58.0,33.0
4,CYP,Cyprus,NaN,NaN,NaN,NaN,NaN,70.0
5,CZE,Czech Republic,57.0,58.0,57.0,74.0,70.0,29.0
6,DNK,Denmark,18.0,74.0,16.0,23.0,35.0,70.0
7,EST,Estonia,40.0,60.0,30.0,60.0,82.0,16.0
8,FIN,Finland,33.0,63.0,26.0,59.0,38.0,57.0
9,FRA,France,68.0,71.0,43.0,86.0,63.0,48.0


### Scrape the EUI Democracy Index (2019) from the Wikipedia page:

[https://en.wikipedia.org/wiki/Democracy_Index]

In [969]:
import bs4
import requests
import country_converter as coco

In [1050]:
 def fetch_democracy_index(save_file=True,
                              threshold=30,
                              data_path='assets/',
                              filename='EUI_democracyIndex_EU.csv'):
    if check_file_age(filename, threshold=threshold):
            return pd.read_csv(f'{data_path}{filename}')

    di_url = 'https://en.wikipedia.org/wiki/Democracy_Index'
    r = requests.get(di_url)
    soup = bs4.BeautifulSoup(r.content, 'html.parser')
    tables = soup.find_all("table")
    dict_list = []
    for idx, table in enumerate(tables):
        if ((table.find('caption') is not None) and
            (table.find('caption').get_text() == 'Democracy Index 2019\n')):
                rows = table.find_all('tr')
                for rdx, row in enumerate(rows[1:168]):
                    row_dict = {}
                    columns = row.find_all('td')
                    try:
                        row_dict['DI_Rank'] = int(columns[0].get_text())
                        offset = 1
                    except:
                        row_dict['DI_Rank'] = rdx - 1
                        offset = 0
                    row_dict['DI_Country'] = columns[0+offset].get_text().strip('\xa0')
                    row_dict['DI_Overall'] = float(columns[1+offset].get_text())
                    row_dict['DI_EPP'] = float(columns[2+offset].get_text())
                    row_dict['DI_FoG'] = float(columns[3+offset].get_text())
                    row_dict['DI_PP'] = float(columns[4+offset].get_text())
                    row_dict['DI_PC'] = float(columns[5+offset].get_text())
                    row_dict['DI_CL'] = float(columns[6+offset].get_text())
                    row_dict['DI_Regime'] = columns[7+offset].get_text()
                    #row_dict['DI_Region'] = columns[8+offset].get_text().strip('\n')
                    dict_list.append(row_dict)
    di_df = pd.DataFrame(dict_list)

    eu_states = fetch_eu_states()
    di_df = pd.merge(eu_states, di_df, how='left',
                     left_on='CountryName', right_on='DI_Country')
    
    sel_cols = ['ISO3166_a3', 'CountryName', 'DI_Rank', 'DI_Overall', 'DI_EPP',
                'DI_FoG', 'DI_PP', 'DI_PC', 'DI_CL', 'DI_Regime']
    di_df = di_df[sel_cols]
    
    di_df.columns = ['ISO3166_a3', 'CountryName', 'DI_Rank', 'DI_Overall', 'DI_EPP',
                'DI_FoG', 'DI_PP', 'DI_PC', 'DI_CL', 'DI_Regime']
    if save_file:
        di_df.to_csv(f"{data_path}{filename}", index=False)

    return di_df

In [1051]:
di_df = fetch_democracy_index()
di_df

>> File [ EUI_democracyIndex_EU.csv ] not found. Fetching...


,ISO3166_a3,CountryName,DI_Rank,DI_Overall,DI_EPP,DI_FoG,DI_PP,DI_PC,DI_CL,DI_Regime
0,AUT,Austria,16,8.29,9.58,7.86,8.33,6.88,8.82,Full democracy
1,BEL,Belgium,33,7.64,9.58,8.21,5.00,6.88,8.53,Flawed democracy
2,BGR,Bulgaria,47,7.03,9.17,6.43,7.22,4.38,7.94,Flawed democracy
3,HRV,Croatia,59,6.57,9.17,6.07,5.56,5.00,7.06,Flawed democracy
4,CYP,Cyprus,34,7.59,9.17,6.43,6.67,6.88,8.82,Flawed democracy
5,CZE,Czech Republic,32,7.69,9.58,6.79,6.67,6.88,8.53,Flawed democracy
6,DNK,Denmark,7,9.22,10.00,9.29,8.33,9.38,9.12,Full democracy
7,EST,Estonia,27,7.90,9.58,7.86,6.67,6.88,8.53,Flawed democracy
8,FIN,Finland,5,9.25,10.00,8.93,8.89,8.75,9.71,Full democracy
9,FRA,France,20,8.12,9.58,7.86,7.78,6.88,8.53,Full democracy


### World Bank Indicators


It appears the press freedom is using a similar, but slightly different API. Oh the joys...

https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019

In [961]:
import urllib

In [962]:
def wb_request_indicator(indicator, country=None, mrnev=None,
						 date=None):
	# Adapted from here: [ https://datahelpdesk.worldbank.org/knowledgebase/articles/898581-api-basic-call-structures ]
	if country is None:
		country = 'all'
	elif type(country) is list:
		country = ";".join(country).lower()
	wb_request_url = f"https://api.worldbank.org/v2/country/{country}/indicator/{indicator}?format=json"

	if mrnev is not None:
		wb_request_url += f'&mrnev={str(mrnev)}'
	elif date is not None:
		if type(date) is list:
			wb_request_url += f'&date={date_range[0]}:{date_range[1]}'
		else:
			wb_request_url += f'&date={date}'
	#
	print(f"Request URL [ {wb_request_url} ]")
	with urllib.request.urlopen(wb_request_url) as response:
		response_json = json.loads(response.read())

	if 'page' in response_json[0].keys():
		page = response_json[0]['page']
		total_pages = response_json[0]['pages']
		indicator_list = response_json[1]

		while page != total_pages:
			with urllib.request.urlopen(wb_request_url + f'&page={page+1}') as response:
				response_json = json.loads(response.read())
			indicator_list += response_json[1]
			page = response_json[0]['page']

		indicator_list = list(map(flatten_wb_indicator, indicator_list))
		return pd.DataFrame(indicator_list)
	else:
		print("Error. Invalid request:")
		print(f"Request [{wb_request_url}]")
		print(f"Response:")
		print(response_json)
		return None

In [963]:
def fetch_indicators(indicators,
					save_file=True,
					data_path='assets/',
					filename='wb_indicators.csv',
					threshold=30):
	if check_file_age(filename, threshold=threshold):
		return pd.read_csv(f'{data_path}{filename}')

	mrnev = 1
	indicator_df = pd.DataFrame()
	for indicator in indicators:
		request_df = wb_request_indicator(indicator, mrnev=1)
		indicator_df = pd.concat([indicator_df, request_df])
		#print(indicator_df.shape[0])

	if save_file:
		indicator_df.to_csv(f'{data_path}{filename}', index=False)
	return indicator_df

In [964]:
def flatten_wb_indicator(item):
	item['indicator_id'] = item['indicator']['id']
	item['indicator_name'] = item['indicator']['value']
	item['country_id'] = item['country']['id']
	item['country_name'] = item['country']['value']
	item.pop('indicator')
	item.pop('country')
	return item

In [967]:
# indicator_names = {"SP.POP.TOTL" : "population",
# 					"NY.GDP.MKTP.CD" : "gdp_total",
# 					"SP.DYN.LE00.MA.IN" : "lifeExp_male",
# 					"SP.DYN.LE00.FE.IN" : "lifeExp_female",
#                     "h3f86901f" : "pressFreedom"}
indicator_names = {'h3f86901f':'pressFreedom'}

indicator_df = fetch_indicators(list(indicator_names.keys()))

>> File [ wb_indicators.csv ] not found. Fetching...
Request URL [ https://api.worldbank.org/v2/country/all/indicator/h3f86901f?format=json&mrnev=1 ]
Error. Invalid request:
Request [https://api.worldbank.org/v2/country/all/indicator/h3f86901f?format=json&mrnev=1]
Response:
[{'message': [{'id': '120', 'key': 'Invalid value', 'value': 'The provided parameter value is not valid'}]}]


#### ToDo:

+ Restructure this to the standard format for the APIs

In [966]:
indicator_df

,countryiso3code,date,value,obs_status,decimal,indicator_id,indicator_name,country_id,country_name
0,ARB,2018,4.197906e+08,NaN,0,SP.POP.TOTL,"Population, total",1A,Arab World
1,CSS,2018,7.358965e+06,NaN,0,SP.POP.TOTL,"Population, total",S3,Caribbean small states
2,CEB,2018,1.025306e+08,NaN,0,SP.POP.TOTL,"Population, total",B8,Central Europe and the Baltics
3,EAR,2018,3.249140e+09,NaN,0,SP.POP.TOTL,"Population, total",V2,Early-demographic dividend
4,EAS,2018,2.328138e+09,NaN,0,SP.POP.TOTL,"Population, total",Z4,East Asia & Pacific
...,...,...,...,...,...,...,...,...,...
1024,VIR,2018,8.300000e+01,NaN,0,SP.DYN.LE00.FE.IN,"Life expectancy at birth, female (years)",VI,Virgin Islands (U.S.)
1025,PSE,2018,7.559500e+01,NaN,0,SP.DYN.LE00.FE.IN,"Life expectancy at birth, female (years)",PS,West Bank and Gaza
1026,YEM,2018,6.779500e+01,NaN,0,SP.DYN.LE00.FE.IN,"Life expectancy at birth, female (years)",YE,"Yemen, Rep."
1027,ZMB,2018,6.644700e+01,NaN,0,SP.DYN.LE00.FE.IN,"Life expectancy at birth, female (years)",ZM,Zambia


### Merge everything

In [1278]:
def fetch_constants(save_file=True,
                        threshold=30,
                        data_path='assets/',
                        filename='CountryFixedEffects.csv'):
    if check_file_age(filename, threshold=threshold):
        return pd.read_csv(f'{data_path}{filename}')
    indicators = {'PAGGTOPY':{'unit':'PERSMYNB','name':'Phys_total'},
              'MINUINFI':{'unit':'PERSMYNB','name':'Nurse_total'},
              'HOPITBED':{'unit':'NOMBRENB','name':'HospBed_total'},} 

    final_df = fetch_eu_states()[['ISO3166_a3', 'CountryName']]
    eurostat_df = fetch_eurostat_indicators().drop('CountryName', axis=1)
    oecd_indicators = get_oecd_indicators(indicators).drop('CountryName', axis=1)
    hofstede_df = load_cultural_dimensions_data().drop('CountryName', axis=1)
    di_df = fetch_democracy_index().drop('CountryName', axis=1)
    
    final_df = pd.merge(final_df, eurostat_df, how='left', on='ISO3166_a3')
    final_df = pd.merge(final_df, hofstede_df, how='left', on='ISO3166_a3')
    final_df = pd.merge(final_df, oecd_indicators, how='left', on='ISO3166_a3')
    final_df = pd.merge(final_df, di_df, how='left', on='ISO3166_a3')
    
    if save_file:
        final_df.to_csv(f"{data_path}{filename}", index=False)
    
    return final_df

In [1280]:
final_df = fetch_constants()
final_df

>> File  [ CountryFixedEffects.csv ] found. Checking age... 
	+ File  [ CountryFixedEffects.csv ] not expired. Load from disk...


,ISO3166_a3,CountryName,EUS_EUS_healthExp_pc_gdp_2016,EUS_EUS_healthExp_eur_hab_2016,EUS_EUS_population_2019,EUS_EUS_govt_budget_surplus_pct_gdp_2019,EUS_EUS_govt_debt_pct_gdp_2019,EUS_EUS_population_density_2018,EUS_EUS_popOver65_pct_2019,EUS_EUS_singleHouseholds_pct_2018,...,OECD_Nurse_total_2017,OECD_HospBed_total_2017,DI_Rank,DI_Overall,DI_EPP,DI_FoG,DI_PP,DI_PC,DI_CL,DI_Regime
0,AUT,Austria,10.42,4248.37,38250.0,0.7,70.4,107.1,18.8,16.8,...,60263.0,64805.00,16,8.29,9.58,7.86,8.33,6.88,8.82,Full democracy
1,BEL,Belgium,10.30,3861.23,35900.0,-1.9,98.6,375.3,18.9,15.3,...,NaN,64423.00,33,7.64,9.58,8.21,5.00,6.88,8.53,Flawed democracy
2,BGR,Bulgaria,8.23,555.64,6800.0,2.1,20.4,63.9,21.3,13.8,...,NaN,NaN,47,7.03,9.17,6.43,7.22,4.38,7.94,Flawed democracy
3,HRV,Croatia,6.83,763.03,12480.0,0.4,73.2,73.2,20.6,8.9,...,NaN,NaN,59,6.57,9.17,6.07,5.56,5.00,7.06,Flawed democracy
4,CYP,Cyprus,6.79,1474.00,24250.0,1.7,95.5,94.4,16.1,7.8,...,NaN,NaN,34,7.59,9.17,6.43,6.67,6.88,8.82,Flawed democracy
5,CZE,Czech Republic,7.15,1193.39,18000.0,0.3,30.8,137.7,19.6,12.2,...,85372.0,70214.00,32,7.69,9.58,6.79,6.67,6.88,8.53,Flawed democracy
6,DNK,Denmark,10.18,5014.00,49190.0,3.7,33.2,138.0,19.6,22.2,...,NaN,15037.00,7,9.22,10.00,9.29,8.33,9.38,9.12,Full democracy
7,EST,Estonia,6.50,1071.71,15670.0,-0.3,8.4,30.4,19.8,18.6,...,8157.0,6185.00,27,7.90,9.58,7.86,6.67,6.88,8.53,Flawed democracy
8,FIN,Finland,9.43,3707.20,37170.0,-1.1,59.4,18.1,21.8,21.5,...,NaN,18071.90,5,9.25,10.00,8.93,8.89,8.75,9.71,Full democracy
9,FRA,France,11.48,3843.67,33270.0,-3.0,98.1,105.6,20.1,16.5,...,NaN,399865.00,20,8.12,9.58,7.86,7.78,6.88,8.53,Full democracy
